# Set up

In [ ]:
%pip install pandas
%pip install pygal
%pip install cairosvg
%pip install matplotlib
%pip install seaborn
%pip install sklearn
%pip install scikit-learn

## Imports

In [ ]:
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler


## Import and load spreadsheet

In [ ]:
data = '230607data.csv'
# headers = data.pop(0)
df = pd.read_csv(data, header=0)

In [ ]:
demographic_columns = ['po _1','gender','residence','country','vegetarian','vegan']
cat_columns = ['Caring About Animals', 'Support for Institutional Welfare Solutions','Perceptions of Meat & Plant-Based Food','Perceptions of Intersecting Issues','Support for Farmed Animal Advocacy','Consumption of Animal Products (Self-Reported)']
cat_columns_reorder = ['Caring About Animals', 'Support for Institutional Welfare Solutions',
               'Knowledge about Farmed Animal Conditions', 'Consumption of Animal Products (Self-Reported)',
               'Perceptions of Meat & Plant-Based Food','Support for Farmed Animal Advocacy',
               'Perceptions of Intersecting Issues']
df = df.reindex(columns=demographic_columns+cat_columns_reorder)

In [ ]:
for col in cat_columns_reorder:
  df[col] = pd.to_numeric(df[col])

# for col in demographic_columns:
  # df[col] = pd.to_numeric(df[col])


# Exploratory Analysis

## Demographic data

In [ ]:
def barplot(col):
  
  mean = df.groupby(col)[columns_to_cluster].mean()
  std = df.groupby(col)[columns_to_cluster].std()
  ax = mean.plot(kind='bar')
  ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

## Histograms of cat_columns

In [ ]:
for col in cat_columns_reorder:
    plt.figure()
    sns.histplot(df[col],kde=False)
    plt.show()

In [ ]:
# Data Exploration
print(df.head())
print(df.info())
print(df.describe())

## Paired scatterplots

In [ ]:
x_values = [float(row[col_1]) for row in data[1:]]
y_values = [float(row[col_2]) for row in data[1:]]

# mean and std
x_mean, y_mean = np.mean(x_values), np.mean(y_values)
x_std, y_std = np.std(x_values), np.std(y_values)
print(x_mean, x_std, y_mean, y_std)

#density coloring
xy = np.vstack([x_values, y_values])
z = gaussian_kde(xy)(xy)

plt.scatter(x_values, y_values, marker='x', c=z, cmap='plasma')
plt.errorbar(x_mean, y_mean, xerr=x_std, yerr=y_std, fmt='o', color='red')

# plt.colorbar()
x = str(data[0][col_1])
y = str(data[0][col_2])
plt.xlabel(x)
plt.ylabel(y)
plt.title('Scatter plot')
# plt.show()
plt.savefig('/content/drive/My Drive/MFA/Plots/scatterplots2/' + x + ',' + y)

In [8]:
columns_to_plot = df[cat_columns_reordered]
sns.set(style="ticks")
sns.pairplot(columns_to_plot, diag_kind="kde", plot_kws={'alpha':0.6})
plt.show()


NameError: name 'cat_columns_reordered' is not defined

## Paired comparisons

In [ ]:
# Pairplot for cat columns
sns.pairplot(df[cat_columns])
plt.savefig('/content/drive/MyDrive/MFA/pairplot'+''+'.png')

In [ ]:
# Correlation heatmap for cat columns
corr = df[cat_columns].corr()
print(corr)
print(corr.shape)
sns.heatmap(corr, annot=True, cmap='coolwarm',center='0')
plt.savefig('/content/drive/MyDrive/MFA/correlation_heatmap'+''+'.png',bbox_inches='tight')

# Clustering Algorithms

## k-means clustering

In [ ]:
# K-means Elbow chart
inertia = []

# Specify the range of 'k' values to test
k_values = range(1, 11)

# Perform clustering for each value of 'k'
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df[cat_columns])
    inertia.append(kmeans.inertia_)

# Plot the elbow chart
plt.plot(k_values, inertia, marker='o')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Sum of Squared Distances')
plt.title('Elbow Chart')
plt.show()

In [ ]:
# def k_cluster(n):

# Select the columns you want to cluster and convert to numeric
columns_to_cluster = cat_columns_reorder
df[columns_to_cluster] = df[columns_to_cluster].apply(pd.to_numeric)

# Perform KMeans clustering
kmeans = KMeans(n_clusters=4)
clusters = kmeans.fit_predict(df[columns_to_cluster])

# Add the cluster labels to the dataframe
df['cluster'] = clusters
print(df)

In [ ]:
means = df.groupby('cluster')[columns_to_cluster].mean()
stds = df.groupby('cluster')[columns_to_cluster].std()

## Gaussian Mixture

# Current clustering method

In [ ]:

# Assuming you have your data stored in the 'data' variable
data = df[cat_columns_reorder]
# Create the GMM object
gmm = GaussianMixture(n_components=3)  # Specify the number of components/clusters

# Fit the GMM model to the data
gmm.fit(data)

# Obtain the cluster labels assigned by GMM
labels = gmm.predict(data)

# Print the results
print(labels)

# Create a DataFrame to store the cluster labels
cluster_df = pd.DataFrame(labels, columns=['cluster'])

# Count the occurrences of each cluster label
cluster_counts = cluster_df['cluster'].value_counts()

# Sort the cluster counts in descending order
cluster_counts_sorted = cluster_counts.sort_values(ascending=False)

# Create a mapping dictionary for the new cluster labels
cluster_mapping = {cluster_counts_sorted.index[i]: i for i in range(len(cluster_counts_sorted))}

# Map the new cluster labels to the original DataFrame
df['cluster'] = cluster_df['cluster'].map(cluster_mapping)

# print
cluster_counts = df['cluster'].value_counts()
print(cluster_counts)


In [ ]:
cluster_counts = df['cluster'].value_counts()
print(cluster_counts)
cluster_counts = df['cluster_reassigned'].value_counts()
print(cluster_counts)

In [ ]:
# Step 1: Filter rows with cluster label '1'
cluster_1_df = df[df['cluster'] == 0]

# Step 2: Extract relevant features, if needed
cluster_1_df = cluster_1_df[cat_columns_reorder]

# Step 3: Apply clustering algorithm
kmeans = KMeans(n_clusters=2)  # You can adjust the number of clusters as needed
subcluster_labels = kmeans.fit_predict(cluster_1_df)

# Map the subcluster labels from 0 and 1 to 3 and 4
subcluster_labels_mapped = subcluster_labels + 3

# Step 4: Assign subcluster labels back to the original DataFrame
df.loc[df['cluster'] == 0, 'subcluster'] = subcluster_labels_mapped
# Step 4: Assign subcluster labels back to the original DataFrame
df.loc[df['cluster'] == 0, 'cluster_reassigned'] = subcluster_labels_mapped
df.loc[df['cluster'] != 0, 'cluster_reassigned'] = df.loc[df['cluster'] != 0, 'cluster']

# Print the updated DataFrame


# Count

In [ ]:

cluster_counts = df['cluster_reassigned'].value_counts()
print(cluster_counts)

# Cluster Demographics

In [ ]:
columns_to_plot=demographic_columns

for column in columns_to_plot:
    # Group the dataframe by the 'cluster' column and the current column
    grouped = df.groupby([column, 'cluster_reassigned']).size().unstack()
    
    # Normalize so all bars are the same total length
    grouped = grouped.div(grouped.sum(axis=1), axis=0)

    # Plot the grouped data as a bar chart
    grouped.plot(kind='barh', stacked=True)
    
    # Set the title and labels
    plt.title(f'Bar Chart for {column} by Cluster')
    plt.xlabel('Cluster')
    plt.ylabel('Count')
    
    # Display the plot
    plt.show()

In [ ]:
for cluster in df['cluster_reassigned'].unique():
    # Filter the dataframe for the current cluster
    cluster_df = df[df['cluster_reassigned'] == cluster]
    
    for column in columns_to_plot:
        # Group the filtered dataframe by the current column
        plt.figure()
        grouped = cluster_df.groupby(column).size()
        
        # Plot the grouped data as a bar chart
        grouped.plot(kind='bar', stacked=True)
        
        # Set the title and labels
        plt.title(f'Bar Chart for {column} in Cluster {cluster}')
        plt.xlabel('Count')
        plt.ylabel(column)
        
        # Move the legend out of the way
        # plt.ylim(0, 14000)

        plt.legend().set_visible(False)
        
        # Display the plot
        plt.savefig(f'230614/barcharts/{column} in Cluster {cluster}.png')


In [ ]:
print(means)
print(stds)

# Cluster characteristic graphs

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import colorsys
import numpy as np

# Define the columns to plot
columns_to_plot = cat_columns_reorder
print(columns_to_plot)

# Create the line chart
fig, ax = plt.subplots()

# Loop through each cluster and create a line chart
count = 0
clusters = df['cluster_reassigned'].unique()
for cluster in clusters:
    # Filter the dataframe to only include rows in the current cluster
    subset = df[df['cluster_reassigned'] == cluster]

    # Get the means and standard deviations for the subset
    means = subset[columns_to_plot].mean()
    stds = subset[columns_to_plot].std()
    x = range(len(columns_to_plot))

    # hue = count / len(clusters)
    # count += 1
    # color = colorsys.hsv_to_rgb(hue, 0.8, 0.8)

    # Plot the means with error bars
    ax.errorbar(x, means, marker='o', label='Cluster ' + str(cluster))

mean = df[columns_to_plot].mean()

# Add the overall means
ax.errorbar(x, mean, marker='x', label='overall mean', color='gray', linestyle='dotted')

# Set the x-axis ticks and labels
ax.set_xticks(x)
ax.set_xticklabels(columns_to_plot, rotation=90)

# Set the title and labels
plt.title('Line Chart for Clusters')
plt.xlabel('Columns')
plt.ylabel('Mean')

# Add a legend

plt.legend()

# Display the plot
# plt.savefig('230614/sanegraph.png',bbox='tight')


## Radar chart

In [ ]:
from IPython.display import display
import pygal
from pygal import Config
from pygal.style import Style
import pandas as pd

In [ ]:
%pip install pillow

In [ ]:
import colorsys
import pygal


In [ ]:
# def radar_chart(columns_to_plot):

# Define the columns to plot
columns_to_plot = cat_columns_reorder
print(columns_to_plot)


# Create the radar chart
chart = pygal.Radar(fill=False)
chart.x_labels = columns_to_plot
chart.range = [0,2]


# Loop through each cluster and create a radar chart
count = 0
clusters = df['cluster'].unique()
for cluster in clusters:
    # Filter the dataframe to only include rows in the current cluster
    subset = df[df['cluster'] == cluster]
    
    # Get the means for the subset
    means = subset[columns_to_plot].mean()
    # q1s = subset[columns_to_plot].quantile([0.25])
    # print(means)

    hue = count / len(clusters)
    count += 1
    color = colorsys.hsv_to_rgb(hue, 0.8, 0.8)
    chart.add('Cluster ' + str(cluster), [round(mean, 2)+1 for mean in means], stroke_style={'width': 10, 'color': color})
    # chart.add('Cluster ' + str(cluster), [round(q1, 2)+1 for q1 in q1s], stroke_style={'width': 10, 'color': color})
    

chart.render_to_file('Plots/spider4/'+'clusters_kmeans_' + str(len(clusters
                                                                                       )) +'_' + str(cluster) + '.svg')


    # chart_data = chart.render_to_png()
    # display(SVG(chart.render()))
    # Convert the PNG data to a base64-encoded string
    # chart_data_base64 = base64.b64encode(chart_data).decode('utf-8')

    # Display the chart as an HTML image
    # display(HTML('<img src="data:image/png;base64,{0}">'.format(chart_data_base64)))

In [ ]:
# def radar_chart(columns_to_plot):

# Define the columns to plot
columns_to_plot = cat_columns_reorder
print(columns_to_plot)


# Loop through each cluster and create a radar chart
count = 0
clusters = df['cluster'].unique()
for cluster in clusters:
    # Filter the dataframe to only include rows in the current cluster
    subset = df[df['cluster'] == cluster]

    # Create the radar chart
    chart = pygal.Radar(fill=False)
    chart.x_labels = columns_to_plot
    chart.range = [0,2.4]
        
    # Get the means for the subset
    means = subset[columns_to_plot].mean()
    stds = subset[columns_to_plot].std()
    lowers = means-std
    uppers = means + std

    # hue = count / len(clusters)
    # count += 1
    # color = colorsys.hsv_to_rgb(hue, 0.8, 0.8)
    chart.add('Cluster ' + str(cluster), [round(mean, 2)+1 for mean in means], stroke_style={'width': 10})
    chart.add('Cluster -std' + str(cluster), [round(lower, 2)+1 for lower in lowers], stroke_style={'width': 10})
    chart.add('Cluster +std' + str(cluster), [round(upper, 2)+1 for upper in uppers], stroke_style={'width': 10})
    

    chart.render_to_file('Plots/spider5/'+'clusters_kmeans_' +'_' + str(cluster) + '.svg')

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# columns_to_cluster = cat_columns

# pca = PCA(n_components=2)
# pca_result = pca.fit_transform(df[columns_to_cluster])

# print("Explained variance ratio 2:", pca.explained_variance_ratio_)


# # plt.scatter(pca_result[:,0], pca_result[:,1], c=df['cluster'], cmap='viridis')
# # plt.xlabel('PC1')
# # plt.ylabel('PC2')
# # plt.title('PCA Plot of Clusters')
# # plt.show()
# n_components = 7
# pca = PCA(n_components)
# pca_result = pca.fit_transform(df[columns_to_cluster])

# print("Explained variance ratio 3:", pca.explained_variance_ratio_)

In [ ]:
loadings = []

for i in range(n_components):

  loading = pd.DataFrame(np.abs(pca.components_[i]))
  loadings.append(loading)

loadings = np.squeeze(loadings)
# Print the loadings
print(loadings)
sns.heatmap(loadings, annot=True, yticklabels=['PC-'+str(x) for x in range(7)], cmap='coolwarm', center=0)
# plt.show()
plt.savefig('/Users/siaosilooi/Library/CloudStorage/GoogleDrive-looisiaosi@gmail.com/My Drive/MFA/loadings_heatmap_PCAcomponents'+''+'.png',bbox_inches='tight',)

In [ ]:
# Correlation heatmap
pca_df = pd.DataFrame(pca_result)
# print(pca_df.head())
# print(pca_df.info())
# print(pca_df.describe())

dfs=[]
for i in range(6):
  corrwith = pd.DataFrame(df[cat_columns]).corrwith(pca_df[i])
  corr_df = pd.DataFrame(np.reshape(corrwith.values,(1,-1)),columns=corrwith.index)
  dfs.append(corr_df)
  
corr = pd.concat(dfs, axis=0)
print(corr)
print(corr.shape)
# corr.plot(kind='barh')
plt.figure()


sns.heatmap(corr, annot=True, yticklabels=['PC-'+str(x) for x in range(6)],cmap='coolwarm')
plt.show()
# plt.savefig('/content/drive/MyDrive/MFA/correlation_heatmap_PCAcomponents'+''+'.png',bbox_inches='tight')

In [ ]:
from kmodes.kmodes import KModes

*italicized text*# Autoencoding

# Autoencoding

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
df2 = df[cat_columns_reorder]
# Rescale the values in the 'cat_columns' column
for i in cat_columns_reorder:
  df2[i] = scaler.fit_transform(df[[i]])

In [ ]:
df[cat_columns_reorder]

In [ ]:
df_cat = df2[cat_columns_reorder]

input_dim = len(df_cat.columns)
latent_dim = 1

#create encoder
encoder_inputs = keras.Input(shape=(input_dim,))
x = layers.Dense(16, activation="relu")(encoder_inputs)
x = layers.Dense(32, activation="relu")(x)
latent = layers.Dense(latent_dim, activation="relu")(x)

encoder = keras.Model(encoder_inputs, latent, name="encoder")

#create decoder
decoder_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(16, activation="relu")(decoder_inputs)
x = layers.Dense(32, activation="relu")(x)
decoder_outputs = layers.Dense(input_dim, activation="sigmoid")(x)

decoder = keras.Model(decoder_inputs, decoder_outputs, name="decoder")

#create autoencoder
autoencoder_inputs = keras.Input(shape=(input_dim,))
latent = encoder(autoencoder_inputs)
decoder_outputs = decoder(latent)
autoencoder = keras.Model(autoencoder_inputs, decoder_outputs, name="autoencoder")

# Compile the model

autoencoder.compile(optimizer="adam", loss="binary_crossentropy")

# Train the autoencoder
autoencoder.fit(df_cat, df_cat, epochs=50, batch_size=128, validation_split=0.2)

# Generate reconstructed data from test data
reconstructed = autoencoder.predict(df_cat)

#print
np.set_printoptions(suppress=True)

print('Original data:')
print(df_cat.head())
print('Reconstructed data:')
print(pd.DataFrame(reconstructed, columns=df_cat.columns).head())

In [ ]:
arr = encoder.predict(df_cat)
print(arr)
# df2[e  ncoded_values'] = arr
# Get the highest value
max_val = np.amax(arr)
print("Highest value:", max_val)

# Get the lowest value 

min_val = np.amin(arr)
print("Lowest value:", min_val)

In [ ]:
autoencoder.summary()

In [ ]:
i = 5
samples = np.array([[x/i for x in range(0,math.ceil(max_val)*i)]])
print([x/i for x in range(0,math.ceil(max_val)*i)])
samples = samples.T
# print(samples)
decoder_data = decoder.predict(samples)
print(decoder_data)

In [ ]:
data_t = decoder_data.T
# print(data_t)
plt.figure()

for i in range(data_t.shape[0]):
    plt.plot(samples,data_t[i])
    

plt.legend(cat_columns_reorder,bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()
# plt.savefig()

## Autoencoder plots

In [ ]:
# Assuming your dataset is in the same order as the encoder inputs
encoder_inputs = df[cat_columns_reorder]  # Adjust this if necessary


# Get the encoded output
encoded_output = encoder.predict(encoder_inputs)
# print(encoded_output)
# print(encoded_output.T[0][1])

# Add the encoded output to the DataFrame
df['encoded_output'] = encoded_output.T[0]

# Plotting
plt.figure()

# Iterate over unique clusters
clusters = df['cluster'].unique()
for cluster in clusters:
    cluster_data = df[df['cluster'] == cluster]
    print(1)
    sns.histplot(data=cluster_data['encoded_output'], label=f'Cluster {cluster}')


# Add labels and title
plt.xlabel('Encoded Output')
plt.ylabel('Density')
plt.title('Density Plot of Clusters')

# Add legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
# Assuming your DataFrame is named df
cluster_counts = df['cluster'].value_counts()
print(cluster_counts)


# Cluster plots

In [ ]:

for i in cat_columns_reorder:
    # Plotting
    plt.figure()

    # Iterate over unique clusters
    clusters = df['cluster_reassigned'].unique()
    for cluster in clusters:
        cluster_data = df[df['cluster_reassigned'] == cluster]
        sns.histplot(data=cluster_data[i], label=f'Cluster {cluster}', kde=False, bins=20)

    # Add labels and title
    plt.xlabel(i)
    plt.ylabel('Count')
    plt.title('Count Plot of Clusters')

    # Add legend
    plt.legend()

    # Show the plot
    plt.show()


In [ ]:
# Assuming your dataset is in the same order as the encoder inputs
encoder_inputs = df[cat_columns_reorder]  # Adjust this if necessary

# Get the encoded output
encoded_output = encoder.predict(encoder_inputs)

# Plotting
plt.figure(figsize=(10, 6))

# Plot the density of the encoded output
sns.histplot(data=encoded_output, fill=True)

# Add labels and title
plt.xlabel('Encoded Output')
plt.ylabel('Density')
plt.title('Density Plot of Encoded Output')

# Show the plot
plt.show()